# Topic 2: Recommendation System
### Demo: Underthesea, Gemsim, Cosin Similarity

In [1]:
!pip install gensim
!pip install underthesea


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
import re

In [25]:
STOP_WORD_FILE = 'data/vietnamese-stopwords.txt'

In [26]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

# Underthesea
#### word_tokenize, pos_tag, sent_tokenize
- link: https://github.com/undertheseanlp/underthesea

In [27]:
# Dữ liệu thử nghiệm
df= pd.read_csv('data/Product.csv')

In [28]:
df.head()

,Unnamed: 0,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh
0,0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/p/r/prom...
1,1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml 95ml 400ml Làm Sạch Sâu Tươi Mát Da Ki...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,https://media.hcdn.vn/catalog/product/f/a/face...
2,2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml 236ml 473ml Da khô/Hỗn hợp khô Da dầu/Hỗn...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9,https://media.hcdn.vn/catalog/product/p/r/prom...
3,3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml 50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...
4,4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml 50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...


In [29]:
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Có thể 1 cột hoặc nhiều cột thông tin
# ví dụ ở đây là chọn 1 cột, và chỉ lấy 200 từ đầu tiên trong phần mô tả (dễ quan sát khi làm demo)
df['Content'] = df['mo_ta'].apply(lambda x: ' '.join(x.split()[:200]))

In [30]:
df.head()

,Unnamed: 0,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh,Content
0,0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/p/r/prom...,Nước Hoa Hồng Klairs Supple Preparation là dòn...
1,1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml 95ml 400ml Làm Sạch Sâu Tươi Mát Da Ki...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
2,2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml 236ml 473ml Da khô/Hỗn hợp khô Da dầu/Hỗn...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9,https://media.hcdn.vn/catalog/product/p/r/prom...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...
3,3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml 50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...
4,4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml 50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...


In [31]:
# word_tokenize
df["Content_wt"]=df["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [32]:
df[["Content", "Content_wt"]].head(2)

,Content,Content_wt
0,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
1,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy_Trang L'Oréal là dòng sản_phẩm tẩy_tr...


In [33]:
import re

In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [35]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,...,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195
0,1.000000,0.046224,0.044385,0.035963,0.069073,0.058979,0.018244,0.058979,0.014660,0.046224,...,0.029718,0.052168,0.024368,0.046224,0.034852,0.012772,0.018331,0.010464,0.057483,0.044285
1,0.046224,1.000000,0.054702,0.205833,0.324406,0.248365,0.345414,0.248365,0.037592,1.000000,...,0.054728,0.136256,0.047468,1.000000,0.114905,0.169409,0.225810,0.041887,0.199865,0.342939
2,0.044385,0.054702,1.000000,0.062374,0.036029,0.074197,0.031340,0.074197,0.035007,0.054702,...,0.037309,0.040818,0.028154,0.054702,0.031271,0.028053,0.054186,0.024236,0.041080,0.046524
3,0.035963,0.205833,0.062374,1.000000,0.380509,0.112106,0.285743,0.112106,0.213317,0.205833,...,0.141310,0.350224,0.024376,0.205833,0.295795,0.203455,0.090484,0.034415,0.167831,0.201528
4,0.069073,0.324406,0.036029,0.380509,1.000000,0.153016,0.397225,0.153016,0.188483,0.324406,...,0.137565,0.321318,0.026869,0.324406,0.333488,0.247697,0.117139,0.031367,0.219160,0.285389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,0.012772,0.169409,0.028053,0.203455,0.247697,0.094678,0.260981,0.094678,0.250544,0.169409,...,0.094364,0.181660,0.008514,0.169409,0.206081,1.000000,0.062103,0.034916,0.105654,0.155256
1192,0.018331,0.225810,0.054186,0.090484,0.117139,0.146091,0.125432,0.146091,0.018781,0.225810,...,0.044507,0.053854,0.078693,0.225810,0.055303,0.062103,1.000000,0.036740,0.074146,0.128376
1193,0.010464,0.041887,0.024236,0.034415,0.031367,0.070780,0.034404,0.070780,0.017479,0.041887,...,0.008222,0.032768,0.007512,0.041887,0.033255,0.034916,0.036740,1.000000,0.018485,0.042118
1194,0.057483,0.199865,0.041080,0.167831,0.219160,0.075644,0.188993,0.075644,0.077838,0.199865,...,0.082509,0.107084,0.029555,0.199865,0.104416,0.105654,0.074146,0.018485,1.000000,0.175580


In [36]:
# Hàm đề xuất sản phẩm
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations_(ma_san_pham, cosine_sim=cosine_sim, nums=3):
    idx = df.index[df['ma_san_pham'] == ma_san_pham][0]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy 3 sản phẩm tương tự nhất
    hotel_indices = [i[0] for i in sim_scores]
    return df.iloc[hotel_indices]

In [37]:
# Gọi function
recommendations = get_recommendations_(318900012)
recommendations

0


,Unnamed: 0,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh,Content,Content_wt
97,97,318900011,Nước Hoa Hồng Klairs Dành Cho Da Nhạy Cảm 180ml,217000,409000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
598,596,422203798,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,60000,135000.0,NaN,Nước Hoa Hồng Klairs Supple Preparation Unscen...,0.0,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation Unscen...,Nước Hoa_Hồng Klairs_Supple Preparation_Unscen...
67,67,211300002,Nước Hoa Hồng Simple Làm Dịu Da & Cấp Ẩm 200ml,109000,180000.0,NaN,Nước Hoa Hồng Simple Kind to Skin Soothing Fac...,4.8,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Simple Kind to Skin Soothing Fac...,Nước Hoa_Hồng Simple Kind to Skin_Soothing Fac...


#### Lưu ma trận kết quả consin và đọc lên khi cần đề xuất

In [38]:
# Save cosine_sim to file
import pickle
with open('products_cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

# Open and read file to cosine_sim_new
with open('products_cosine_sim.pkl', 'rb') as f:
    cosine_sim_new = pickle.load(f)
# cosine_sim_new

In [39]:
def get_recommendations(df, ma_san_pham, cosine_sim=cosine_sim, nums=5):
    # Get the index of the hotel that matches the hotel_id
    matching_indices = df.index[df['ma_san_pham'] == ma_san_pham].tolist()
    if not matching_indices:
        print(f"No hotel found with ID: {ma_san_pham}")
        return pd.DataFrame()  # Return an empty DataFrame if no match
    idx = matching_indices[0]

    # Get the pairwise similarity scores of all hotels with that hotel
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the hotels based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar hotels (Ignoring the hotel itself)
    sim_scores = sim_scores[1:nums+1]

    # Get the hotel indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top n most similar hotels as a DataFrame
    return df.iloc[product_indices]

In [40]:
# Example usage (make sure to use a valid ma_san_pham from your DataFrame)
recommendations = get_recommendations(df, 318900012, cosine_sim=cosine_sim_new, nums=3)
recommendations

,Unnamed: 0,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh,Content,Content_wt
97,97,318900011,Nước Hoa Hồng Klairs Dành Cho Da Nhạy Cảm 180ml,217000,409000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
598,596,422203798,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,60000,135000.0,NaN,Nước Hoa Hồng Klairs Supple Preparation Unscen...,0.0,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation Unscen...,Nước Hoa_Hồng Klairs_Supple Preparation_Unscen...
67,67,211300002,Nước Hoa Hồng Simple Làm Dịu Da & Cấp Ẩm 200ml,109000,180000.0,NaN,Nước Hoa Hồng Simple Kind to Skin Soothing Fac...,4.8,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Simple Kind to Skin Soothing Fac...,Nước Hoa_Hồng Simple Kind to Skin_Soothing Fac...
